In [21]:
import quandl
import pandas as pd
import ssl
import pickle
import matplotlib.pyplot as plt
from matplotlib import style

style.use('fivethirtyeight')

api_key = "9BxGjBmVKT_CGXQ6Vk-x"



In [ ]:

def states_list():   
    ssl._create_default_https_context = ssl._create_unverified_context
    state_list = pd.read_html('https://simple.wikipedia.org/wiki/List_of_U.S._states')
    return state_list[0][0][1:]
    
def grab_state_data():
    
    states = states_list()
    main_df = pd.DataFrame()

    for abbv in states:
        query = "FMAC/HPI_"+str(abbv)   #the api names to query for!
        df = quandl.get(query, authtoken=api_key)
        df.rename(columns={'Value':str(abbv)}, inplace=True)
        #df = df.pct_change()      ##percent change
        ### percent change from first year
        df[abbv] = (df[abbv] - df[abbv][0]) / df[abbv][0] * 100.0
        
        if main_df.empty:
            main_df = df
        else:
            main_df = main_df.join(df)

    print(main_df.head())
    
    #pickle queries
    pickle_out = open('us_states2.pickle', 'wb')
    pickle.dump(main_df, pickle_out)
    pickle_out.close()
  
#grab_state_data()


                  AL        AK        AZ        AR        CA        CO  \
Date                                                                     
1975-01-31  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
1975-02-28  0.588316  1.482897  1.544274  0.927908  0.325085  1.027497   
1975-03-31  1.290141  3.000603  2.996426  1.727446  1.520570  2.072651   
1975-04-30  2.167505  4.584117  4.111578  2.337585  3.528148  3.080431   
1975-05-31  3.015661  6.313363  4.687991  2.868375  5.215956  3.924871   

                  CT        DE         FL        GA    ...           SD  \
Date                                                   ...                
1975-01-31  0.000000  0.000000   0.000000  0.000000    ...     0.000000   
1975-02-28  2.167603  0.096771   3.941241 -0.963826    ...    -1.085751   
1975-03-31  3.797352  0.296394   9.788003 -1.389000    ...    -2.191044   
1975-04-30  4.696220  0.756425  16.956330 -1.198660    ...    -3.289066   
1975-05-31  4.940457  1.581382 

In [24]:
def HPI_benchmark():
    df = df = quandl.get("FMAC/HPI_USA", authtoken=api_key)
    df.rename(columns={'Value':"United States"}, inplace=True)
    df["United States"] = (df["United States"] - df["United States"][0]) / df["United States"][0] * 100.0
    return df

In [37]:
fig = plt.figure()
ax1 = plt.subplot2grid((1, 1), (0, 0))

#pickle_in = open('us_states.pickle', 'rb')

HPI_data = pd.read_pickle('us_states2.pickle')

TX1yr = HPI_data['TX'].resample('A').ohlc() #resample() for changing sampling rate
print(TX1yr.head())

HPI_data['TX'].plot(ax=ax1, label='Monthly TX HPI')
TX1yr.plot(ax=ax1, label='yearly TX HPI')

plt.legend(loc=4)
plt.show()

##generate correlation table for all the columns
# HPI_correlation = HPI_data.corr()
# print(HPI_correlation)
# print(HPI_correlation.describe())

                 open       high        low      close
Date                                                  
1975-12-31   0.000000   6.163697   0.000000   5.836106
1976-12-31   6.224214  14.212953   6.224214  14.212953
1977-12-31  14.774936  29.370158  14.774936  29.370158
1978-12-31  30.756707  49.599720  30.756707  49.599720
1979-12-31  51.019921  74.488959  51.019921  74.488959
